# Priestley-Taylor (etw) Example

In [16]:

import ee
import openet.refetgee

ee.Initialize()


### Load GRIDMET for a single date

In [17]:
gridmet_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
    .filterDate('2018-04-01', '2018-04-02')
    
gridmet_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
gridmet_img = ee.Image('IDAHO_EPSCOR/GRIDMET/20180401')

### Ancillary data and test point

In [18]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match GRIDMET elevation grid exactly
elev_img = ee.Image("projects/climate-engine/gridmet/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the GRIDMET images
crs = gridmet_img.projection().getInfo()['wkt']
geo = gridmet_img.projection().getInfo()['transform']

### Precomputer ETr

In [19]:
etr = gridmet_img.select('etr').reduceRegion(
    ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

7.800000190734863


### Compute ETw for a single image

In [20]:
print(openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).etw\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etw'])

3.3535871712894094


### Compute ETw for a collection in a mapped function

In [21]:
def gridmet_etw(img):
    return ee.Image(openet.refetgee.Daily.gridmet(img, elev=elev_img, lat=lat_img).etw)\
        .copyProperties(img, ['system:time_start'])
etw = ee.Image(ee.ImageCollection(gridmet_coll.map(gridmet_etw)).first())\
    .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etw.getInfo()['etw'])

3.3535871712894094
